p 85~120

# 6. 정수 인코딩 (Integer Encoding)
- 텍스트->숫자로 바꾸는 기법
- 각 단어를 고유한 정수(인덱스)에 맵핑시키는 전처리 작업 필요
- 보통은 단어 빈도수 기준 정렬 후 인덱스 부여

6-1) 정수 인코딩
- 빈도수 순으로 정렬한 단어 집합 생성
- 빈도수 높은 순서대로 낮은 정수 부여

6-1-1) dictionary 사용해 정수 인코딩

In [1]:
# dictionary 사용하는 방법

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
raw_text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# 문장 토큰화
sentences = sent_tokenize(raw_text) # NLTK의 영어 문장 단위 토큰화 메소드
print(sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# 정제, 정규화하며 단어 토큰화 / 단어 소문자화 -> 단어 개수 통일 / 불용어 단어 길이 2 이하 -> 일부 제외
# 텍스트 수치화 = 본격적인 자연어 처리 작업 직전 -> 단어가 텍스트일 때만 할 수 있는 전처리 끝내야 함

vocab = {} # 빈 딕셔너리 선언 (단어 : 숫자 맵핑 할 딕셔너리)
preprocessed_sentences = [] # 빈 배열 선언
stop_words = set(stopwords.words('english'))

for sentence in sentences: # 문장 순회하며
    # 단어 토큰화
    tokenized_sentence = word_tokenize(sentence) # 문장 단위 토큰화 한 것을 단어 단위로 토큰화
    result = [] # 불용어 제거한 단어 (=전처리된 결과) 담을 리스트

    for word in tokenized_sentence: # 현재 문장 내 단어 토큰들 순회하며
        word = word.lower() # 소문자화 -> 단어의 개수 줄임
        if word not in stop_words: # 불용어 아니고 (= 불용어 제거)
            if len(word) > 2: # 단어 길이가 2이하가 아니면  (= 길이 2 이하인 단어 제거)
                result.append(word) # 전처리된 단어 리스트에 추가
                if word not in vocab: # 단어 빈도 딕셔너리 vocab에 없는 단어이면
                    vocab[word] = 0  # 저장, 빈도 초기화
                vocab[word] += 1 # vocab에 있는 단어이면 빈도 +1
    preprocessed_sentences.append(result) # 문장별로 전처리된 문장 리스트 preprocessed_sentences에 추가

print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [8]:
print(vocab) # 단어와 빈도수가 딕셔너리 형태로 저장됨

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


In [9]:
print(vocab["barber"]) # 'barber'라는 단어의 빈도수 출력

8


- 빈도수 높은 순서대로 정렬

In [10]:
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [11]:
word_to_index = {}
i = 0
for (word, frequency) in vocab_sorted :
    if frequency > 1 : # 빈도수가 작은 단어(= 빈도수가 1인 단어)는 제외.
        i = i + 1
        word_to_index[word] = i

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [12]:
vocab_size = 5
words_frequency = [word for word, index in word_to_index.items() if index >= vocab_size + 1]
# 인덱스가 5 초과인 단어 제거 = 빈도수 상위 5개 단어만 보존

for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


- OOV(Out-Of-Vocabulary) 문제 : 단어 집합에 없는 단어 문제
- word_to_index에 'OOV'라는 단어 추가 -> 단어 집합에 없는 단어가 입력으로 들어올 경우 OOV의 인덱스로 인코딩

In [13]:
word_to_index['OOV'] = len(word_to_index)+1 # 'OOV'의 인덱스를 맨 마지막 인덱스+1로 지정
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


In [14]:
# 단어와 인덱스가 쌍으로 저장된 word_to_index 딕셔너리 이용하여 정수 인코딩

encoded_sentences = [] # 정수 인코딩 된 문장 저장할 리스트
for sentence in preprocessed_sentences: # 전처리된 문장 리스트 순회
    encoded_sentence = [] # 각 문장의 정수 인코딩 결과 저장할 리스트
    for word in sentence: # 현재 문장 내 단어 순회
        try:
          # 단어 집합에 있는 단어이면 해당 단어의 정수 리턴
            encoded_sentence.append(word_to_index[word]) # 현재 단어 정수 인코딩 결과 encoded_sentence에 저장
        except KeyError:
          # 단어집합에 없는 단어이면 'OOV'의 정수 리턴
            encoded_sentence.append(word_to_index['OOV'])
    encoded_sentences.append(encoded_sentence) # 현재 문장 정수 인코딩 결과 encoded_sentences에 저장
print(encoded_sentences)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


6-1-2) Counter 사용하여 정수 인코딩

In [15]:
from collections import Counter # 각 원소의 빈도수 출력, python 제공

In [16]:
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [17]:
# words = np.hstack(preprocessed_sentences)으로도 수행 가능. np.hstack(): 배열 결합
all_words_list = sum(preprocessed_sentences, []) # 하나의 리스트로 평탄화(flatten)
print(all_words_list)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


In [18]:
# 파이썬의 Counter 모듈을 이용하여 단어의 빈도수 카운트
vocab = Counter(all_words_list)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [19]:
print(vocab["barber"]) # 'barber'라는 단어의 빈도수 출력

8


In [20]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
#.most_common() : 빈도 순으로 정렬된 항목의 리스트 반환
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [21]:
word_to_index = {}
i = 0
for (word, frequency) in vocab : # word : 단어, frequency : 빈도수
    i = i + 1 # 인덱스+1
    word_to_index[word] = i # 인덱스 i 부여

print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


6-1-3) NLTK의 FreqDist 사용해 정수 인코딩

In [22]:
from nltk import FreqDist # FreqDist : 주어진 텍스트에서 단어의 빈도 측정에 사용
import numpy as np

In [23]:
vocab = FreqDist(np.hstack(preprocessed_sentences)) # preprocessed_sentences를 평탄화 후 빈도 측정 -> 딕셔너리 형태로 저장됨

In [24]:
vocab

FreqDist({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, ...})

In [25]:
print(vocab["barber"]) # 'barber'라는 단어의 빈도수 출력

8


In [26]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]


In [27]:
word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)}
# enumerate로 vocab의 index와 word를 동시에 가져옴
# index는 0부터 정해지기 때문에 정수 인코딩 시 +1을 해서 저장
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


6-1-4) enumerate : 리스트의 인덱스와 값 동시에 가져오기

In [28]:
test_input = ['a', 'b', 'c', 'd', 'e']
for index, value in enumerate(test_input): # 입력의 순서대로 0부터 인덱스를 부여함.
  print("value : {}, index: {}".format(value, index))

value : a, index: 0
value : b, index: 1
value : c, index: 2
value : d, index: 3
value : e, index: 4


6-2) Keras의 텍스트 전처리
- Keras : 전처리 도구 제공
- Keras의 토크나이저로 정수 인코딩 하기도 함

In [29]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [30]:
preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

In [31]:
# 1. 모든 토큰에 대해 인덱싱 하는 경우

tokenizer = Tokenizer()

# fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성.
tokenizer.fit_on_texts(preprocessed_sentences) # 각 단어에 높은 빈도수 순으로 고유 인덱스 할당

In [32]:
print(tokenizer.word_index) # 부여된 고유한 인덱스 담고 있는 딕셔너리

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [33]:
print(tokenizer.word_counts) # 각 단어의 빈도수 담고 있는 딕셔너리

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [34]:
print(tokenizer.texts_to_sequences(preprocessed_sentences)) # 텍스트를 정수 시퀀스로 변환

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [35]:
print(preprocessed_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [36]:
# 2. 상위 몇 개 토큰만 사용할지 지정

vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1) # 상위 5개 단어만 사용
# most_common() 대신, Tokenizer(num_words= 숫자)로 상위 몇개의 단어 사용할 것인지 지정
# num_words 매개변수는 0부터 카운트 -> 우리는 1부터 인덱싱 했기 때문에 5개 단어를 보존하려면 5+1까지 보존해야 함
tokenizer.fit_on_texts(preprocessed_sentences) # 토큰화, 인덱스 지정

In [37]:
print(tokenizer.word_index)
# 5개를 지정했음에도 전부 다 출력되는 이유 : texts_to_sequences를 사용할 때 적용되기 때문

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [38]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [39]:
print(tokenizer.texts_to_sequences(preprocessed_sentences))
# 1-5번 인덱스에 해당하지 않는 단어는 삭제됨

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


# 7. 패딩(Padding)
- 기계의 병렬 연산을 위해, 여러 문장의 길이를 임의로 동일하게 맞추는 작업

7-1) Numpy로 패딩하기

In [40]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer # 정수 시퀀스로 변환 -> 벡터화, 기본 공백 기준 구분

In [41]:
preprocessed_sentences = [['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

In [42]:
tokenizer = Tokenizer() # 토큰화
tokenizer.fit_on_texts(preprocessed_sentences) # 빈도수 순 단어 집합 생성
encoded = tokenizer.texts_to_sequences(preprocessed_sentences) # 각 단어를 인코딩한 정수대로 맵핑
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [43]:
# 문장 길이 맞추기 위해 가장 긴 문장의 길이 확인
max_len = max(len(item) for item in encoded)
print('최대 길이 :',max_len)

최대 길이 : 7


In [50]:
# 모든 문장의 길이를 7로 맞추기 -> 7보다 짧은 문장은 0을 채워 길이 맞추기
for sentence in encoded: # 인코딩 된 문장 순회
    while len(sentence) < max_len: # 현재 순회중인 문장 길이가 최장길이 7보다 짧으면 (길어지면 루프 탈출)
        sentence.append(0) # 0 삽입

padded_np = np.array(encoded) # 패딩된 인코딩 문장 리스트 encoded를 np.array 형식으로 변환
padded_np
# 패딩 처리(데이터에 특정 값을 채워 shape를 조정하는 것)로 병렬처리 할 수 있는 상태가 됨, 실제 자연어 처리 과정에서 0은 무시됨
# 제로 패딩 : 0으로 패딩처리 하는 것

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

In [51]:
print(sentence)

[1, 12, 3, 13, 0, 0, 0]


In [52]:
encoded

[[1, 5, 0, 0, 0, 0, 0],
 [1, 8, 5, 0, 0, 0, 0],
 [1, 3, 5, 0, 0, 0, 0],
 [9, 2, 0, 0, 0, 0, 0],
 [2, 4, 3, 2, 0, 0, 0],
 [3, 2, 0, 0, 0, 0, 0],
 [1, 4, 6, 0, 0, 0, 0],
 [1, 4, 6, 0, 0, 0, 0],
 [1, 4, 2, 0, 0, 0, 0],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13, 0, 0, 0]]

7-2) 케라스 전처리 도구로 패딩
- pad_sequences()

In [53]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

encoded = tokenizer.texts_to_sequences(preprocessed_sentences) # 정수 인코딩 된 배열로 만듦
print(encoded)

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


In [54]:
padded = pad_sequences(encoded) # 앞이 0으로 채워짐, 뒤에 채우고 싶으면 padding='post'로 설정하면 됨
padded

array([[ 0,  0,  0,  0,  0,  1,  5],
       [ 0,  0,  0,  0,  1,  8,  5],
       [ 0,  0,  0,  0,  1,  3,  5],
       [ 0,  0,  0,  0,  0,  9,  2],
       [ 0,  0,  0,  2,  4,  3,  2],
       [ 0,  0,  0,  0,  0,  3,  2],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  2],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 0,  0,  0,  1, 12,  3, 13]], dtype=int32)

In [55]:
padded = pad_sequences(encoded, padding='post')
padded

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]], dtype=int32)

In [56]:
(padded == padded_np).all()
# pad_sequences()로 패딩한 것과, 순회하며 0을 append 한 후 np.array로 변환한 것 결과가 정확히 일치함

True

In [57]:
# 가장 긴 문장이 아닌, 임의의 길이(5)에 맞추어 패딩하는 경우
# 5보다 긴 문장은  padding='post'인 경우 뒷부분 손실됨
padded = pad_sequences(encoded, padding='post', maxlen=5)
padded

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0]], dtype=int32)

In [58]:
# 0이 아닌 다른 숫자로 패딩하는 경우
# 현재는 정수 인코딩 된 숫자와 겹치지 않도록, 단어 집합 크기+1한 숫자로 패딩
last_value = len(tokenizer.word_index) +1
print(last_value)

14


In [59]:
padded = pad_sequences(encoded, padding = 'post', value=last_value)
padded

array([[ 1,  5, 14, 14, 14, 14, 14],
       [ 1,  8,  5, 14, 14, 14, 14],
       [ 1,  3,  5, 14, 14, 14, 14],
       [ 9,  2, 14, 14, 14, 14, 14],
       [ 2,  4,  3,  2, 14, 14, 14],
       [ 3,  2, 14, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  2, 14, 14, 14, 14],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13, 14, 14, 14]], dtype=int32)

# 8. 원-핫 인코딩 (One-Hot Encoding)
- 단어 집합 : 서로 다른 단어들의 집합, 같은 단어의 변형 형태도 다른 단어로 간주
- 원핫인코딩 준비 과정 : 단어 집합 생성 -> 정수 인코딩

8-1) 원-핫 인코딩이란?
- 원핫인코딩 : 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 값에 1, 나머지에 0부여하는 단어의 벡터 표현 방식

In [62]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 9.2 MB/s eta 0:00:00


In [63]:
# 한국어 문장을 원핫인코딩 하는 예제
# 1. Okt(open korean text) 이용해 토큰화
from konlpy.tag import Okt # 한국어 토큰화 모듈

okt = Okt()
tokens = okt.morphs("나는 자연어 처리를 배운다") # morphs() : 형태소 반환 (토큰)
print(tokens)

['나', '는', '자연어', '처리', '를', '배운다']


In [64]:
# 각 토큰에 정수 부여 (짧은 문장이기 때문에 빈도수 순 정렬 과정 생략)
word_to_index = {word : index for index, word in enumerate(tokens)} # 토큰 value와 index 가져와 딕셔너리 형태로 저장
print('단어 집합 :',word_to_index)

단어 집합 : {'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [65]:
# 토큰 입력하면 원핫벡터 만들어내는 함수
def one_hot_encoding(word, word_to_index):
  one_hot_vector = [0]*len(word_to_index)
  index = word_to_index[word]
  one_hot_vector[index] = 1
  return one_hot_vector

In [66]:
one_hot_encoding('자연어', word_to_index)

[0, 0, 1, 0, 0, 0]

8-2) 케라스를 이용한 원-핫 인코딩
- to_categorical()

In [67]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

In [70]:
# 단어 집합 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text]) # fit_on_texts() : 빈도수 기준 단어 집합 생성, 정수 인코딩
# 맵핑하려면 texts_to_sequences() 사용
print('단어 집합 :', tokenizer.word_index)

단어 집합 : {'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


In [71]:
tokenizer

In [72]:
# 정수 인코딩
sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"
encoded = tokenizer.texts_to_sequences([sub_text])[0]
encoded

[2, 5, 1, 6, 3, 7]

In [74]:
# 원핫인코딩
one_hot = to_categorical(encoded)
print(one_hot)

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


8-3) 원-핫 인코딩의 한계
- 단어가 늘어날수록 벡터 저장 공간 (차원) 늘어남 (단어 집합의 크기 = 벡터 차원임)
- 단어의 유사도 표현 불가 -> 이를 해결하기 위해 단어의 잠재 의미 반영하여 다차원 공간에 벡터화 하는 기법 있음

1) 카운트 기반 벡터화 방법
- LSA(잠재 의미 분석), HAL

2) 예측 기반 벡터화
- NNLM, RNNLM, Word2Vec, FastText

3) 카운트 기반 + 예측 기반
- GloVe

# 9. 데이터 분리 (Splitting Data)
- 지도학습(Supervised learning)을 위한 데이터 분리

In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

9-1) X, y 분리
- zip 함수 : 동일한 개수를 가지는 시퀀스 자료형에서, 각 순서에 등장하는 원소끼리 묶어줌 / 리스트의 리스트에서 X, y 분리하는데 유용
- 데이터프레임 이용
- numpy 이용 : 슬라이싱

In [76]:
# 1. zip 함수 이용해 분리
X, y = zip(['a', 1], ['b', 2], ['c', 3])
print('X 데이터 :',X)
print('y 데이터 :',y)

X 데이터 : ('a', 'b', 'c')
y 데이터 : (1, 2, 3)


In [79]:
sequences = [['a', 1], ['b', 2], ['c', 3]]
X, y = zip(*sequences) # * 없으면 'ValueError: too many values to unpack (expected 2)'오류 발생 = 변수가 정해진 개수보다 많이 입력됨
# * : 언패킹 연산자. 리스트, 튜플 등의 iterable한(= 반복 가능한) 객체 언패킹하여 각 요소를 개별 변수로 전달
# *를 통해 a와1 / b와 2 / c와 3이 전달됨 -> zip으로 같은 위치 원소끼리 묶임
print('X 데이터 :',X)
print('y 데이터 :',y)

X 데이터 : ('a', 'b', 'c')
y 데이터 : (1, 2, 3)


In [83]:
# 2. 데이터프레임 이용하여 분리
values = [['당신에게 드리는 마지막 혜택!', 1],
['내일 뵐 수 있을지 확인 부탁드...', 0],
['철수씨. 잘 지내시죠? 오랜만입...', 0],
['(광고) AI로 주가를 예측할 수 있다!', 1]]

columns = ['메일 본문', '스팸 메일 유무']

df = pd.DataFrame(values, columns=columns)
df

,메일 본문,스팸 메일 유무
0,당신에게 드리는 마지막 혜택!,1
1,내일 뵐 수 있을지 확인 부탁드...,0
2,철수씨. 잘 지내시죠? 오랜만입...,0
3,(광고) AI로 주가를 예측할 수 있다!,1


In [81]:
# 컬럼 지정 안하고 데이터프레임으로 만든 경우
df2 = pd.DataFrame(values)
df2

,0,1
0,당신에게 드리는 마지막 혜택!,1
1,내일 뵐 수 있을지 확인 부탁드...,0
2,철수씨. 잘 지내시죠? 오랜만입...,0
3,(광고) AI로 주가를 예측할 수 있다!,1


In [84]:
X = df['메일 본문']
y = df['스팸 메일 유무']

In [85]:
print('X 데이터 :',X.to_list())
print('y 데이터 :',y.to_list())

X 데이터 : ['당신에게 드리는 마지막 혜택!', '내일 뵐 수 있을지 확인 부탁드...', '철수씨. 잘 지내시죠? 오랜만입...', '(광고) AI로 주가를 예측할 수 있다!']
y 데이터 : [1, 0, 0, 1]


In [86]:
# 3. Numpy 이용해 분리 : 슬라이싱
np_array = np.arange(0,16).reshape((4,4)) # 임의 데이터 생성 : 0~15범위의 수로 4*4 데이터 생성
print('전체 데이터 :')
print(np_array)

전체 데이터 :
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


In [95]:
X = np_array[:,:3]
y = np_array[:,3] # y = np_array[:, : 3]로 하면, y의 각 원소가 다른 배열에 저장됨

print('X :', X)
print('y :', y)

X : [[ 0  1  2]
 [ 4  5  6]
 [ 8  9 10]
 [12 13 14]]
y : [ 3  7 11 15]


In [96]:
X1 = np_array[:,:3]
y1 = np_array[:,3:]

print('X1 :', X1)
print('y1 :', y1) # 이 형태이면 data split에서 오류 남

X1 : [[ 0  1  2]
 [ 4  5  6]
 [ 8  9 10]
 [12 13 14]]
y1 : [[ 3]
 [ 7]
 [11]
 [15]]


9-2) 테스트 데이터 분리
- 사이킷 런 이용 : train_test_split()
- 수동 분리

In [97]:
# 1. 사이킷 런 이용해 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
# X : 독립 변수, y: 종속변수, test_size : test data의 비율(20%), 데이터 개수로 지정 가능, random_state : 난수 시드
# random_state를 같게 설정하면 동일하게 섞은 후 split 한 데이터 얻을 수 있음


In [98]:
# 임의로 X와 y 데이터를 생성
X, y = np.arange(10).reshape((5, 2)), range(5)

print('X 전체 데이터 :')
print(X)
print('y 전체 데이터 :')
print(list(y))

X 전체 데이터 :
[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
y 전체 데이터 :
[0, 1, 2, 3, 4]


In [99]:
# 7:3의 비율로 훈련 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [100]:
print('X 훈련 데이터 :')
print(X_train)
print('X 테스트 데이터 :')
print(X_test)

X 훈련 데이터 :
[[2 3]
 [4 5]
 [6 7]]
X 테스트 데이터 :
[[8 9]
 [0 1]]


In [101]:
print('y 훈련 데이터 :')
print(y_train)
print('y 테스트 데이터 :')
print(y_test)

y 훈련 데이터 :
[1, 2, 3]
y 테스트 데이터 :
[4, 0]


In [102]:
# random_state 변경해보기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print('y 훈련 데이터 :')
print(y_train)
print('y 테스트 데이터 :')
print(y_test)

y 훈련 데이터 :
[4, 0, 3]
y 테스트 데이터 :
[2, 1]


In [103]:
# random_state 다시 원래대로
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
print('y 훈련 데이터 :')
print(y_train)
print('y 테스트 데이터 :')
print(y_test)

y 훈련 데이터 :
[1, 2, 3]
y 테스트 데이터 :
[4, 0]


In [104]:
# 2. 수동 분리
# 임의로 X와 y 데이터를 생성
X, y = np.arange(0,24).reshape((12,2)), range(12)
print('X 전체 데이터 :')
print(X)
print('y 전체 데이터 :')
print(list(y))

X 전체 데이터 :
[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]
 [20 21]
 [22 23]]
y 전체 데이터 :
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [105]:
# 데이터 분할 비율(갯수) 결정

num_of_train = int(len(X) * 0.8) # 데이터의 전체 길이의 80%에 해당하는 길이값을 구한다.
num_of_test = int(len(X) - num_of_train) # 전체 길이에서 80%에 해당하는 길이를 뺀다.
# num_of_test 를 len(X) * 0.2 로 계산하면 안됨!! 데이터에 누락이 발 생할 수 있음
print('훈련 데이터의 크기 :',num_of_train)
print('테스트 데이터의 크기 :',num_of_test)

훈련 데이터의 크기 : 9
테스트 데이터의 크기 : 3


In [106]:
# 분할 : 데이터를 섞지 않은 채로 분할하게 됨. 수동 분리 시 데이터 섞는 과정 필

X_test = X[num_of_train:] # 전체 데이터 중에서 20%만큼 뒤의 데이터 저장
y_test = y[num_of_train:] # 전체 데이터 중에서 20%만큼 뒤의 데이터 저장
X_train = X[:num_of_train] # 전체 데이터 중에서 80%만큼 앞의 데이터 저장
y_train = y[:num_of_train] # 전체 데이터 중에서 80%만큼 앞의 데이터 저장

In [107]:
# 분할 결과 확인

print('X 테스트 데이터 :')
print(X_test)
print('y 테스트 데이터 :')
print(list(y_test))

X 테스트 데이터 :
[[18 19]
 [20 21]
 [22 23]]
y 테스트 데이터 :
[9, 10, 11]


# 10. 한국어 전처리 패키지 정리
- KoNLPy, KSS 외 패키지

1. PyKoSpacing : 띄어쓰기 되지 않은 문장 띄어쓰기 처리, 높은 정확도

2. Py-Hanspell : 네이버 한글 맞춤법 검사기 기반, 맞춤법 및 띄어쓰기 보정

3. SOYNLP : 품사 태깅, 단어 토큰화 지원하는 단어 토크나이저. 비지도 학습으로 단어 토큰화.
- 데이터에 자주 등장하는 단어 분석, 응집 확률과 브랜칭 엔트로피 활용 -> 기존 형태소 분석기가 가지는 신조어 문제 해결
- 특장 문자열이 자주 연결되어 등장하면 한 단어로 판단, 앞 뒤로 조사 또는 완전히 다른 단어가 자주 등장하면 그러한 단어를 한 단어라고 파악
- 응집 확률 (cohesion probability) : 내부 문자열이 응집하여 얼마나 자주 등장하는지 판단하는 척도 (각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산하여 누적곱을 한 값)
- 브랜칭 엔트로피 (branching entropy) : 다음에 나올 문자의 확률(척도)
- 반복되는 문자 정제 : ㅋㅋㅋ등 불필요하게 연속되는 경우 하나의 단어로 정규화

4. Customized KoNLPy : 사용자 사전 추가 (사람 이름 등)